In [1]:
import random
from math import sqrt
from functools import lru_cache

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.ent = None

    def empty(self):
        return self.ent is None

class Entity:
    def __init__(self, color):
        self.color = color

class Eater(Entity): pass

class Food(Entity):
    def __init__(self, color, is_last=False):
        super().__init__(color)
        self.is_last = is_last

class Wall(Entity):
    def __init__(self):
        super().__init__("Wall")

class Grid:
    DIRS = [(0,1), (0,-1), (1,0), (-1,0)]
    COLOR_MAP = {
        'Food': {'White': 0, 'Green': 1, 'Blue': 2, 'Red': 3, 'Yellow': 4, 'Purple': 5, 'Pink': 6},
        'Eater': {'Green': 0, 'Blue': 1, 'Red': 2, 'Yellow': 3, 'Purple': 4, 'Pink': 5}
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = [[Cell(x, y) for y in range(cols)] for x in range(rows)]
        self.moves = []
        self.min_separation = 2
        self.walls = []  # Store wall positions
        # Add memoization cache for expensive calculations
        self._distance_cache = {}
        self._cluster_cache = {}

    def calculate_clustering_score(self):
        """Calculate how clustered the puzzle is. Lower scores are better (less clustered)."""
        # Clear memoization caches when starting a new calculation
        self._distance_cache = {}
        self._cluster_cache = {}
        
        colors = set(self.COLOR_MAP['Food'].keys()) - {'White'}
        metrics = {}
        
        for color in colors:
            # Get positions of all food items of this color
            foods = [(x, y) for x in range(self.rows) for y in range(self.cols)
                    if isinstance(self.grid[x][y].ent, Food) and self.grid[x][y].ent.color == color]
            
            if not foods:
                continue

            # Get position of matching eater
            eater_pos = next(((x, y) for x in range(self.rows) for y in range(self.cols)
                            if isinstance(self.grid[x][y].ent, Eater) and self.grid[x][y].ent.color == color), None)
            
            if not eater_pos:
                continue

            # Calculate clustering metrics for this color
            color_metrics = {
                'food_count': len(foods),
                'avg_food_separation': self._calculate_avg_separation(foods),
                'largest_cluster_size': self._find_largest_cluster(foods),
                'avg_distance_to_eater': self._calculate_avg_distance_to_point(foods, eater_pos),
                'distribution_score': self._calculate_distribution_score(foods),
                'all_adjacent': self._check_all_adjacent(foods, color)  # New metric
            }
            
            metrics[color] = color_metrics

        # Calculate final score (lower is better)
        if not metrics:
            return {'final_score': float('inf'), 'color_metrics': {}}

        final_score = self._calculate_final_score(metrics)
        
        # Check if any color has all food items adjacent to each other
        has_all_adjacent = any(m.get('all_adjacent', False) for m in metrics.values())
        
        return {
            'final_score': float('inf') if has_all_adjacent else final_score,
            'color_metrics': metrics,
            'has_all_adjacent': has_all_adjacent  # Flag for disqualification
        }

    def _check_all_adjacent(self, positions, color):
        """
        Check if all food items of the same color form a single connected group.
        Returns True if all items are adjacent to at least one other item of the same color.
        """
        if len(positions) <= 1:
            return False
            
        # Use BFS to determine if all food items are connected
        visited = set()
        queue = [positions[0]]
        visited.add(positions[0])
        
        while queue:
            x, y = queue.pop(0)
            for dx, dy in self.DIRS:
                nx, ny = x + dx, y + dy
                if (nx, ny) in positions and (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append((nx, ny))
        
        # If all positions are visited, they form a single connected group
        return len(visited) == len(positions)

    def _manhattan_distance(self, pos1, pos2):
        """Calculate Manhattan distance between two positions with memoization"""
        # Create a consistent key for the cache regardless of parameter order
        key = tuple(sorted([pos1, pos2]))
        if key not in self._distance_cache:
            self._distance_cache[key] = abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
        return self._distance_cache[key]

    def _calculate_avg_separation(self, positions):
        """Calculate average minimum distance between all pairs of positions"""
        if len(positions) < 2:
            return float('inf')
        
        # Create a consistent cache key
        positions_key = tuple(sorted(positions))
        if positions_key in self._cluster_cache:
            return self._cluster_cache[positions_key].get('avg_separation', float('inf'))
            
        distances = []
        for i, pos1 in enumerate(positions[:-1]):
            for pos2 in positions[i+1:]:
                distances.append(self._manhattan_distance(pos1, pos2))
        
        avg_separation = sum(distances) / len(distances) if distances else float('inf')
        
        # Store in cache
        if positions_key not in self._cluster_cache:
            self._cluster_cache[positions_key] = {}
        self._cluster_cache[positions_key]['avg_separation'] = avg_separation
        
        return avg_separation

    def _find_largest_cluster(self, positions, cluster_threshold=2):
        """Find size of largest cluster where points are within cluster_threshold distance"""
        if not positions:
            return 0
            
        # Create a consistent cache key
        positions_key = tuple(sorted(positions))
        cluster_key = f"largest_cluster_{cluster_threshold}"
        
        if positions_key in self._cluster_cache and cluster_key in self._cluster_cache[positions_key]:
            return self._cluster_cache[positions_key][cluster_key]
            
        # Use simple clustering - count points within threshold of each other
        clusters = []
        remaining = set(positions)
        
        while remaining:
            current = remaining.pop()
            cluster = {current}
            to_check = {current}
            
            while to_check:
                pos = to_check.pop()
                nearby = {p for p in remaining 
                         if self._manhattan_distance(pos, p) <= cluster_threshold}
                cluster.update(nearby)
                to_check.update(nearby)
                remaining -= nearby
            
            clusters.append(len(cluster))
        
        largest_cluster = max(clusters) if clusters else 1
        
        # Store in cache
        if positions_key not in self._cluster_cache:
            self._cluster_cache[positions_key] = {}
        self._cluster_cache[positions_key][cluster_key] = largest_cluster
        
        return largest_cluster

    def _calculate_avg_distance_to_point(self, positions, target):
        """Calculate average distance from all positions to target"""
        if not positions:
            return 0
        distances = [self._manhattan_distance(pos, target) for pos in positions]
        return sum(distances) / len(distances)

    def _calculate_distribution_score(self, positions):
        """Calculate how well distributed the positions are across the grid"""
        if not positions:
            return 0
            
        # Create a consistent cache key
        positions_key = tuple(sorted(positions))
        if positions_key in self._cluster_cache and 'distribution' in self._cluster_cache[positions_key]:
            return self._cluster_cache[positions_key]['distribution']
            
        # Divide grid into quadrants and count points in each
        mid_row = self.rows // 2
        mid_col = self.cols // 2
        quadrants = [0] * 4
        
        for x, y in positions:
            quad_idx = (x >= mid_row) * 2 + (y >= mid_col)
            quadrants[quad_idx] += 1
            
        # Calculate standard deviation of quadrant counts
        avg = sum(quadrants) / 4
        variance = sum((q - avg) ** 2 for q in quadrants) / 4
        distribution_score = sqrt(variance)
        
        # Store in cache
        if positions_key not in self._cluster_cache:
            self._cluster_cache[positions_key] = {}
        self._cluster_cache[positions_key]['distribution'] = distribution_score
        
        return distribution_score

    def _calculate_final_score(self, metrics):
        """Calculate final clustering score from all metrics"""
        scores = []
        for color, color_metrics in metrics.items():
            # If all food items of this color are adjacent, heavily penalize
            if color_metrics.get('all_adjacent', False):
                scores.append(1000)  # Extremely high penalty
                continue
                
            # Lower values are better
            color_score = (
                (3 / color_metrics['avg_food_separation']) +  # More separation is better
                (color_metrics['largest_cluster_size'] / 2) +  # Smaller clusters are better
                (color_metrics['distribution_score']) +  # More even distribution is better
                (1 / (color_metrics['avg_distance_to_eater'] + 1))  # Greater average distance from eater is better
            )
            scores.append(color_score)
        
        return sum(scores) / len(scores) if scores else float('inf')
    
    def place(self, ent, x, y):
        self.grid[x][y].ent = ent
        if isinstance(ent, Wall):
            self.walls.append((x, y))

    def clear(self):
        for row in self.grid:
            for cell in row: cell.ent = None
        self.walls = []
        # Clear caches when resetting the grid
        self._distance_cache = {}
        self._cluster_cache = {}

    def gen_puzzle(self, eaters, white_pct, min_food, wall_count=0):
        """
        eaters: Dict[str, int] - color to count mapping
        white_pct: float - percentage of food to make white
        min_food: int - minimum food per color
        wall_count: int - number of walls to place on the grid
        """
        self.clear()
        
        # First, place walls
        self.place_walls(wall_count)
        
        eater_pos = []  # Track eater positions and info

        # Place eaters in empty cells
        colors = [c for c, n in eaters.items() for _ in range(n)]
        random.shuffle(colors)
        
        for color in colors:
            pos = self.get_empty()
            if not pos: break
            x, y = pos
            e = Eater(color)
            self.place(e, x, y)
            eater_pos.append({'e': e, 'x': x, 'y': y, 'count': 0})

        # Initial phase: Ensure minimum food requirements in rounds
        active_eaters = eater_pos.copy()
        while active_eaters:
            moved_any = False
            # Try one move for each active eater
            for einfo in active_eaters[:]:
                if einfo['count'] >= min_food:
                    active_eaters.remove(einfo)
                    continue
                    
                if self.move_eater(einfo):
                    moved_any = True
                else:
                    active_eaters.remove(einfo)
            
            if not moved_any:
                break

        # Fill remaining space in rounds
        remaining_spaces = self.count_empty()
        active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
        
        while remaining_spaces > 0 and active_eaters:
            moved_any = False
            # Try one move for each active eater
            for einfo in active_eaters[:]:
                if self.move_eater(einfo):
                    moved_any = True
                    remaining_spaces -= 1
                else:
                    active_eaters.remove(einfo)
                    
                if remaining_spaces <= 0:
                    break
            
            if not moved_any:
                break
            
            # Refresh list of active eaters
            active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
            random.shuffle(active_eaters)

        self.whiten_food(white_pct)

    def place_walls(self, count):
        """Place walls randomly on empty cells of the grid"""
        if count <= 0:
            return
            
        # Get all empty cells
        empty_cells = [(x, y) for x in range(self.rows) for y in range(self.cols) 
                        if self.grid[x][y].empty() and 0 < x < self.rows and 0 < y < self.cols]
        
        # Calculate density-based wall count if the grid is large
        grid_size = self.rows * self.cols
        max_walls = min(count, len(empty_cells), int(grid_size * 0.25))  # Cap at 25% of grid
        
        if max_walls <= 0:
            return
            
        wall_positions = random.sample(empty_cells, max_walls)
        
        for x, y in wall_positions:
            self.place(Wall(), x, y)
            
        # Validate that walls don't completely isolate parts of the grid
        if not self.validate_grid_connectivity():
            # If invalid, remove all walls and try again with fewer walls
            for x, y in self.walls:
                self.grid[x][y].ent = None
            self.walls = []
            self.place_walls(int(count * 0.8))  # Reduce by 20% and try again

    def validate_grid_connectivity(self):
        """Check if the grid is still navigable with the current wall placement"""
        if not self.walls:
            return True
            
        # Start from a random empty cell
        empty_cells = [(x, y) for x in range(self.rows) for y in range(self.cols) 
                        if self.grid[x][y].empty()]
        if not empty_cells:
            return False
            
        start = empty_cells[0]
        visited = set()
        queue = [start]
        visited.add(start)
        
        # BFS to find reachable cells
        while queue:
            x, y = queue.pop(0)
            for dx, dy in self.DIRS:
                nx, ny = x + dx, y + dy
                if (self.in_bounds(nx, ny) and 
                    (nx, ny) not in visited and 
                    not isinstance(self.grid[nx][ny].ent, Wall)):
                    visited.add((nx, ny))
                    queue.append((nx, ny))
        
        # Check if all empty cells are reachable
        unvisited = [cell for cell in empty_cells if cell not in visited]
        
        # Allow up to 10% of cells to be unreachable
        return len(unvisited) <= len(empty_cells)
    
    def can_move_eater(self, einfo):
        """Check if an eater has any valid moves available"""
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            if new_x is not None and (new_x != x or new_y != y):
                return True
        return False

    def move_eater(self, einfo):
        """Move an eater to farthest possible position in random direction with spacing constraints"""
        random.shuffle(self.DIRS)
        best_move = None
        best_score = float('-inf')
        
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            
            if new_x is None or (new_x == x and new_y == y):
                continue
            
            # Create a temporary board state to evaluate this potential move
            temp_food_position = (x, y)
            
            # Get positions of existing food with this color
            existing_foods = [(fx, fy) for fx in range(self.rows) for fy in range(self.cols)
                           if isinstance(self.grid[fx][fy].ent, Food) 
                           and self.grid[fx][fy].ent.color == einfo['e'].color]
            
            # Add the potential new food position
            test_positions = existing_foods + [temp_food_position]
            
            # Check if this would create a fully connected component
            # We want to avoid moves that would make all foods adjacent
            if len(test_positions) > 1 and self._would_make_all_adjacent(test_positions):
                continue
            
            move_score = self._evaluate_food_placement(x, y, einfo['e'].color)
            
            if move_score > best_score:
                best_score = move_score
                best_move = (new_x, new_y)
        
        # If no good moves were found, try any valid move
        if best_move is None:
            for dx, dy in self.DIRS:
                x, y = einfo['x'], einfo['y']
                new_x, new_y = self.get_max_pos(x, y, dx, dy)
                if new_x is not None and (new_x != x or new_y != y):
                    best_move = (new_x, new_y)
                    break
        
        if best_move:
            new_x, new_y = best_move
            x, y = einfo['x'], einfo['y']
            
            # Check if this is the first food placed for this eater
            is_first_food = einfo['count'] == 0
            
            # Move eater and place food
            self.grid[x][y].ent = None
            self.place(einfo['e'], new_x, new_y)
            # Set is_last=True for first food placed (which will be last eaten)
            self.place(Food(einfo['e'].color, is_last=is_first_food), x, y)
            einfo['count'] += abs(new_x - x) + abs(new_y - y)
            
            self.moves.append({
                'eater_color': einfo['e'].color,
                'start': (new_x, new_y),
                'end': (x, y),
                'food_color': einfo['e'].color,
                'dist': abs(new_x - x) + abs(new_y - y)
            })
            
            einfo['x'], einfo['y'] = new_x, new_y
            return True
            
        return False
    
    def _would_make_all_adjacent(self, positions):
        """
        Check if the positions would form a single connected component
        Used during eater movement to preemptively avoid creating adjacent food clusters
        """
        if len(positions) <= 1:
            return False
            
        # Use BFS to determine if all food items would be connected
        visited = set()
        queue = [positions[0]]
        visited.add(positions[0])
        
        while queue:
            x, y = queue.pop(0)
            for dx, dy in self.DIRS:
                nx, ny = x + dx, y + dy
                if (nx, ny) in positions and (nx, ny) not in visited:
                    visited.add((nx, ny))
                    queue.append((nx, ny))
        
        # If all positions would be visited, they would form a single connected group
        return len(visited) == len(positions)
    
    def _evaluate_food_placement(self, x, y, color):
        """Score a potential food placement position based on separation from same-colored foods."""
        same_color_foods = [(fx, fy) for fx in range(self.rows) for fy in range(self.cols)
                           if isinstance(self.grid[fx][fy].ent, Food) 
                           and self.grid[fx][fy].ent.color == color]
        
        if not same_color_foods:
            return 100  # Perfect score for first food of its color
        
        # Check if this would create an all-adjacent situation
        test_positions = same_color_foods + [(x, y)]
        if self._would_make_all_adjacent(test_positions):
            return -100  # Heavily penalize moves that would make all foods adjacent
        
        # Calculate minimum distance to any same-colored food
        min_distance = min(abs(x - fx) + abs(y - fy) for fx, fy in same_color_foods)
        
        # Score based on distance (higher is better)
        if min_distance >= self.min_separation:
            return 100 - (10 / min_distance)  # Higher score for more separation
        else:
            return 50 - (10 * (self.min_separation - min_distance))  # Lower score for less separation

    def get_max_pos(self, x, y, dx, dy):
        """Get farthest possible position in given direction, stopping at walls"""
        nx, ny = x, y
        while (self.in_bounds(nx + dx, ny + dy) and 
               self.grid[nx + dx][ny + dy].empty() and 
               not isinstance(self.grid[nx + dx][ny + dy].ent, Wall)):
            nx += dx
            ny += dy
            
            # Check if we hit a wall
            if self.in_bounds(nx, ny) and isinstance(self.grid[nx][ny].ent, Wall):
                break
                
        return (nx, ny) if (nx, ny) != (x, y) else (None, None)

    def whiten_food(self, pct):
        # Existing implementation
        food = [(c.x, c.y) for row in self.grid for c in row 
                if isinstance(c.ent, Food) and c.ent.color != 'White' and not c.ent.is_last]
        to_white = random.sample(food, int(len(food) * pct / 100)) if food else []
        for x, y in to_white:
            self.grid[x][y].ent.color = 'White'

    def get_empty(self):
        empty = [(x,y) for x in range(self.rows) for y in range(self.cols) 
                if self.grid[x][y].empty()]
        return random.choice(empty) if empty else None

    def in_bounds(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def count_empty(self):
        return sum(1 for row in self.grid for cell in row if cell.empty())

    def to_scene(self, level_id):
        """Convert grid to Godot scene file"""
        width, height = 1440, 2560
        step = max(180, width // self.cols - 24)
        
        start_x = width/2 - step*self.cols/2 + step*0.5
        start_y = height/2 - step*self.rows/2 + step*0.5 - 40

        scene = f"""[gd_scene load_steps=5 format=3 uid="{level_id}"]

[ext_resource type="Script" path="res://Levels/Level.cs" id="lvlid"]
[ext_resource type="PackedScene" path="res://Entities/Food/Food.tscn" id="foodid"]
[ext_resource type="PackedScene" path="res://Entities/Eater/Eater.tscn" id="eaterid"]
[ext_resource type="PackedScene" path="res://Entities/Wall/Wall.tscn" id="wallid"]

[node name="Level" type="Node"]
script = ExtResource("lvlid")
{self.solution_metadata()}

[node name="Food" type="Node" parent="."]

[node name="Eaters" type="Node" parent="."]

[node name="Walls" type="Node" parent="."]
"""
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                pos_x = start_x + y*step
                pos_y = start_y + x*step
                
                if isinstance(cell.ent, Food):
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = {self.COLOR_MAP['Food'][cell.ent.color]}
IsLast = {"true" if cell.ent.is_last else "false"}
"""
                elif isinstance(cell.ent, Eater):
                    scene += f"""
[node name="Eater{x}_{y}" parent="Eaters" instance=ExtResource("eaterid")]
position = Vector2({pos_x}, {pos_y})
EaterType = {self.COLOR_MAP['Eater'][cell.ent.color]}
BoardStatePositionId = Vector2i({x}, {y})
ValidFoodTypes = [0, {self.COLOR_MAP['Food'][cell.ent.color]}]
"""
                elif isinstance(cell.ent, Wall):
                    scene += f"""
[node name="Wall{x}_{y}" parent="Walls" instance=ExtResource("wallid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
"""
                elif cell.ent is None:
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = 0
IsLast = false
"""
        return scene

    def solution_metadata(self):
        metadata = "metadata/solution = PackedVector2Array("
        for move in self.moves:
                metadata += f"{move["start"][0]}, {move["start"][1]}, {move["end"][0]}, {move["end"][1]}, "
        metadata = metadata[:-2] + ")"
        return metadata

def gen_boards(num_select, total_gen, rows, cols, eaters, white_pct, min_food, wall_count=0):
    """Generate and return top boards with best clustering among completely filled boards"""
    boards = []
    attempts = 0
    max_tries = total_gen * 100

    while len(boards) < total_gen and attempts < max_tries:
        g = Grid(rows, cols)
        g.gen_puzzle(eaters, white_pct, min_food, wall_count)
        
        empty_count = g.count_empty()
        
        # Calculate clustering info first to check for adjacent food disqualification
        clustering_info = g.calculate_clustering_score()
        
        # Only consider boards that:
        # 1. Are completely filled (or have at most 1 empty cell)
        # 2. Meet minimum food requirements
        # 3. Don't have all food of the same color adjacent to each other
        if (empty_count == 0 and 
            all(sum(1 for row in g.grid for cell in row 
                if isinstance(cell.ent, Food) and cell.ent.color == color) >= min_food 
                for color in eaters) and
            not clustering_info.get('has_all_adjacent', False)):
            
            boards.append({
                'board': g,
                'moves': g.moves.copy(),
                'empty': empty_count,
                'clustering_score': clustering_info['final_score'],
                'metrics': clustering_info['color_metrics']
            })
        attempts += 1

    # Sort only by clustering score since we've already filtered for fullness and adjacency
    boards.sort(key=lambda x: x['clustering_score'])
    return boards[:num_select]


# Example usage
if __name__ == "__main__":
    # Updated configuration to include wall_count
    DIFFICULTY = {
        'tutorial': ({'Green': 1}, 3, 3, 2, 30, 1, 0),
        'very_easy': ({'Blue': 1, 'Green': 1}, 4, 4, 3, 30, 2, 0),
        'very_easy2': ({'Blue': 1, 'Green': 1}, 5, 4, 10, 30, 2, 0),
        'easy': ({'Blue': 1, 'Green': 1, 'Red': 1}, 5, 5, 15, 30, 3, 0),
        'easy2': ({'Blue': 1, 'Green': 1, 'Red': 1}, 6, 5, 15, 30, 3, 0),
        'medium': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6, 20, 20, 3, 0),
        'medium2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 20, 20, 3, 1),
        'hard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 20, 20, 4, 1),
        'hard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 20, 20, 4, 2),
        'max': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 20, 20, 5, 2),
        'max2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 20, 20, 5, 3),
        'max3': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 11, 7, 20, 15, 5, 3)
    }

    boards = []
    for diff, (eaters, rows, cols, num_select, white_pct, min_food, wall_count) in DIFFICULTY.items():
        new_boards = gen_boards(num_select, 5000, rows, cols, eaters, white_pct, min_food, wall_count)
        print(f"Generated {len(new_boards)} new boards for difficulty {diff}")
        boards.extend(new_boards)

    # Save boards
    for i, board in enumerate(boards, 1):
        with open(f"../Levels/Level{i}.tscn", "w") as f:
            f.write(board["board"].to_scene(i))

Generated 2 new boards for difficulty tutorial
Generated 3 new boards for difficulty very_easy
Generated 10 new boards for difficulty very_easy2
Generated 15 new boards for difficulty easy
Generated 15 new boards for difficulty easy2
Generated 20 new boards for difficulty medium
Generated 20 new boards for difficulty medium2
Generated 20 new boards for difficulty hard
Generated 20 new boards for difficulty hard2
Generated 20 new boards for difficulty max
Generated 17 new boards for difficulty max2


KeyboardInterrupt: 